# fMRI Demo (premade patterns version)

## Introduction

This is an example of RSA on a BIDS-structured fMRI dataset. This study is still under review, so we will update this demo to include steps to download the data when it becomes publicly available. 

Participants (n=14) viewed images of 16 objects, as well as their written name. Several properties of the stimuli were carefully balanced. They also performed several tasks, but for the purposes of this demo we will ignore these.

![stimuli](demo_fmri_files/mur32_stims.png)

This library depends on *nibabel* for convenience functions to access MRI data, and is used separately in the demo as well, so start by installing nibabel (`pip install nibabel`).

Then run the following imports:

In [ ]:
%matplotlib inline
from os.path import expanduser, join
import json
import numpy
from rsatoolbox.data.dataset import Dataset
from rsatoolbox.data.noise import prec_from_residuals
from rsatoolbox.rdm.calc import calc_rdm
from rsatoolbox.rdm.rdms import concat
from rsatoolbox.vis.rdm_plot import show_rdm
import matplotlib.pyplot as plt

Now we define the path to the BIDS-root data directory

In [ ]:
data_dir = expanduser('~/data/rsatoolbox/mur32/derivatives/nilearn')

Next we use a utility function to find the relevant preprocessed BOLD images:

In [ ]:
print('Loading..')
with open(join(data_dir, 'meta.json')) as fhandle:
    metadata = json.load(fhandle)
data = numpy.load(join(data_dir, 'data.npz'))

In [ ]:

subjects = metadata['subjects']
conditions = metadata['conditions']
dof = metadata['degrees_of_freedom']
N_RUNS = 6

Let's repeat this for the other runs: (takes about 1min per run). Along the way we also store the MSE residuals for each run.

In [ ]:
rdm_list = []
for region_name in metadata['region_names']:
    for sub in subjects:
        print(f'roi {region_name} sub {sub}')

        betas = data[f'betas_sub-{sub}_{region_name}']
        patterns = betas.reshape(-1, betas.shape[-1])
        ds = Dataset(
            measurements=patterns,
            descriptors=dict(sub=sub, roi=region_name),
            obs_descriptors=dict(
                run=numpy.repeat(numpy.arange(N_RUNS), len(conditions)),
                condition=numpy.tile(conditions, N_RUNS)
            )
        )

        runwise_prec_matrix = []
        resids = data[f'resids_sub-{sub}_{region_name}']
        for r in range(N_RUNS):
            runwise_prec_matrix.append(
                prec_from_residuals(
                    resids[r, :, :],
                    dof=dof,
                    method='shrinkage_diag'
                )
            )

        rdm_list.append(
            calc_rdm(
                dataset=ds,
                noise=runwise_prec_matrix,
                method='crossnobis',
                descriptor='condition',
                cv_descriptor='run',
            )
        )
        #raise ValueError
        del rdm_list[-1].descriptors['noise']

In [ ]:
data_rdms = concat(rdm_list)

In [ ]:
roi_rdms = data_rdms.subset('roi', 'fusiform')
fig, _, _ = show_rdm(
    roi_rdms,
    rdm_descriptor='sub',
    show_colorbar='panel'
)

## Model

Let's map the various stimulus properties of interest on a table

In [ ]:
a_ds = run_datasets[0]
obj_conds = numpy.unique(a_ds.obs_descriptors['trial_type']).tolist()
INDOOR = ['bagel', 'candle', 'clock', 'glass', 'kettle', 'knife', 'sponge', 'table']
STRAIGHT = ['candle', 'knife', 'sponge', 'table', 'spade', 'ladder', 'brick', 'pedal']
df = pandas.DataFrame([dict(
    trial_type=c,
    indoor=float(c.split('_')[1] in INDOOR),
    straight=float(c.split('_')[1] in STRAIGHT),
    modality=float('image_' in c)
) for c in obj_conds])
df

We can then turn these into Model RDMs

In [ ]:
model_dataset = Dataset.from_df(df)
model_dataset.channel_descriptors
model_rdms = calc_rdm(
    [model_dataset.split_channel('name')],
    method='euclidean',
    descriptor='trial_type'
)
model_rdms.rdm_descriptors['name'] = model_dataset.channel_descriptors['name']
fig, _, _ = show_rdm(model_rdms, rdm_descriptor='name')
matplotlib.pyplot.show()

Next, we wrap each of the model RDMs in their own fixed `Model` object.

In [ ]:
from rsatoolbox.model.model import ModelFixed

models = []
for model_name in model_rdms.rdm_descriptors['name']:
    model_rdm = model_rdms.subset('name', model_name)
    models.append(ModelFixed(model_name, model_rdm))

## Inference

Let's see how well each of these models explains the fMRI data RDMs

In [ ]:
from rsatoolbox.inference.evaluate import eval_dual_bootstrap

eval_result = eval_dual_bootstrap(models, data_rdms)
print(eval_result)

Next let's plot a comparison of the models:

In [ ]:
from rsatoolbox.vis.model_plot import plot_model_comparison

fig, _, _ = plot_model_comparison(eval_result, sort=True)
matplotlib.pyplot.show()